In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [147]:
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
def get_driver():
    #caps = DesiredCapabilities().CHROME
    #caps["pageLoadStrategy"] = "normal"
    driver = webdriver.Chrome()#desired_capabilities=caps)
    return driver

In [171]:
import re
import ast
gk_stats = ['Squad', 'Appearances', 'PPG', 'Goals', 'Assists', 'Sub on', 'Sub off', 'Yellow cards', 'Seccond Yellow Cards', 'Red cards', 'Goals conceded', 'Clean sheets', 'Minutes played']
field_stats = ['Squad', 'Appearances', 'PPG', 'Goals', 'Assists', 'Own goals', 'Sub on', 'Sub off', 'Yellow cards', 'Seccond Yellow Cards', 'Red cards', 'Penalty goals', 'Minutes per goal', 'Minutes played']

def get_player_stats(player_url, name, driver):
    player = {}
    driver.get(player_url)
    player['Name'] = name
    player_data = driver.find_elements_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span')
    player_data = [data.text for data in player_data]
    for key, value in zip(player_data[::2], player_data[1::2]):
        player[key[:-1]] = value.strip()
    full_stats = driver.find_element_by_xpath("//*[contains(text(),'View full stats')]").get_attribute('href')
    market_value = driver.find_element_by_xpath("//*[contains(text(),'Market value details')]").get_attribute('href')
    driver.get(full_stats.replace('leistungsdaten', 'leistungsdatendetails') + "/saison/2020/plus/1")
    
    stats = driver.find_element_by_xpath('//*[@id="yw1"]/table')
    stats = stats.text.split("\n")
    total = stats[4].split(':   ')[1].split(' ')
    if 'Champions League' in stats[5]:
        player['CL Appearances'] = stats[5].split(' ')[3]
    else:
        player['CL Appearances'] = 0

    if player['Position'] == 'Goalkeeper':
        for key, value in zip(gk_stats, total):
            player[key] = value
    else:
        for key, value in zip(field_stats, total):
            player[key] = value

    driver.get(market_value)
    script = driver.find_element_by_xpath('/html/body/script[9]').get_attribute('innerHTML')
    series = re.search("'series':(.*),'credits'", script).group(1)
    market_values = ast.literal_eval(series)
    player['Market Values'] = market_values
    return player

In [172]:
def accept_cookies(driver):
    wait = WebDriverWait(driver,10)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,'//iframe[@title="SP Consent Message"]')))
    wait.until(EC.element_to_be_clickable((By.XPATH,"//button[contains(@title,'ACCEPT ALL')]"))).click()

In [200]:
import pandas as pd

def get_team(team_link, driver):
    driver.get(team_link)
    wait = WebDriverWait(driver,10)
    players = wait.until(lambda d: d.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr'))
    player_links = [player.find_element_by_class_name('hauptlink').find_elements_by_tag_name('a')[0] for player in players]
    player_links = {player.text: player.get_attribute('href') for player in player_links}
    players = []
    team_name = team_link.split('/')[3]
    print(team_name)
    for name, player_link in list(player_links.items()):
        try:
            players.append(get_player_stats(player_link, name, driver))
        except:
            players.append({'Name': name})
    
    pd.DataFrame(players).to_csv(f"{team_name}.tsv", sep="\t", index=False)
    return players

def get_league(league_link, driver):
    players = []
    driver.get(league_link)
    accept_cookies(driver)
    teams = driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr')
    team_links = [team.find_elements_by_tag_name('td')[2].find_elements_by_tag_name('a')[0].get_attribute('href') for team in teams]
    for team_link in team_links:
        try:
            players += get_team(team_link, driver)
        except:
            continue
    return players
    
def get_country(country_link, driver):
    driver.get(country_link)
    wait = WebDriverWait(driver,10)
    league = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yw1"]/table/tbody/tr[2]/td[1]/table/tbody/tr/td[2]/a')))
    players = get_league(league.get_attribute('href'), driver)
    return players

def get_all():
    players = []
    driver = get_driver()
    driver.get('https://www.transfermarkt.com/wettbewerbe/europa')
    accept_cookies(driver)
    wait = WebDriverWait(driver,10)
    country_map = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="europa_Map"]')))
    countries = country_map.find_elements_by_tag_name('area')
    country_dict = {country.get_attribute('title'): country.get_attribute('href') for country in countries}
    for key, value in country_dict.items():
        print(key)
        country_driver = get_driver()
        if key == 'Montenegro':
            players += get_league(value, country_driver)
        else:
            players += get_country(value, country_driver)
        country_driver.close()
    driver.close()
    return players

In [190]:
players = get_all()
pd.DataFrame(players).to_csv("all.tsv", sep="\t", index=False)

Bosnia & Herzegovina
hsk-zrinjski-mostar
fk-sarajevo
fk-borac-banja-luka
fk-tuzla-city
fk-velez-mostar
fk-zeljeznicar-sarajevo
nk-siroki-brijeg
fk-sloboda-tuzla
hsk-posusje
fk-leotar-trebinje
fk-rudar-prijedor
fk-radnik-bijeljina
Croatia
gnk-dinamo-zagreb
hnk-hajduk-split
nk-osijek
hnk-rijeka
nk-lokomotiva-zagreb
hnk-gorica
hnk-sibenik
slaven-belupo-koprivnica
nk-hrvatski-dragovoljac
nk-istra-1961
Serbia
red-star-belgrade
fk-partizan-belgrade
fk-cukaricki
fk-vojvodina-novi-sad
fk-tsc-backa-topola
fk-spartak-subotica
fk-napredak-krusevac
fk-radnik-surdulica
fk-radnicki-nis
fk-metalac-gornji-milanovac
fk-vozdovac
fk-kolubara-lazarevac
fk-mladost-lucani
fk-proleter-novi-sad
fk-novi-pazar
fk-radnicki-1923-kragujevac
Slovenia
nk-olimpija-ljubljana
nk-maribor
nk-celje
ns-mura
fc-koper
nk-aluminij-kidricevo
nk-domzale
nk-bravo
nk-tabor-sezana
nk-radomlje
Hungary
ferencvarosi-tc
mol-fehervar-fc
puskas-akademia-fc
budapest-honved-fc
kisvarda-fc
ujpest-fc
debreceni-vsc
mtk-budapest
mez%C5%91kove

TimeoutException: Message: timeout: Timed out receiving message from renderer: 298.421
  (Session info: chrome=102.0.5005.115)


In [202]:
def map_club_to_country(club_list):
    driver = get_driver()
    driver.get('https://www.transfermarkt.com/')
    accept_cookies(driver)
    club_country = {}
    for club in club_list:
        search = driver.find_elements_by_xpath('//*[@id="schnellsuche"]/input')
        search[0].clear()
        search[0].send_keys(club)
        search[1].click()
        country = driver.find_element_by_xpath('//*[@id="yw0"]/table/tbody/tr[1]/td[3]/img').get_attribute('title')
        club_country[club] = country
    return club_country

In [213]:
def get_first_league_clubs2020():
    club_country = {}
    driver = get_driver()
    driver.get('https://www.transfermarkt.com/wettbewerbe/europa')
    accept_cookies(driver)
    wait = WebDriverWait(driver,10)
    country_map = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="europa_Map"]')))
    countries = country_map.find_elements_by_tag_name('area')
    country_dict = {country.get_attribute('title'): country.get_attribute('href') for country in countries}
    for key, value in list(country_dict.items()):
        print(key)
        if key != 'Montenegro':
            driver.get(value)
            league = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yw1"]/table/tbody/tr[2]/td[1]/table/tbody/tr/td[2]/a')))
            league_link = league.get_attribute('href')
        else:
            league_link = value
        driver.get(league_link + '/plus/?saison_id=2020')
        teams = driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr')
        team_names = [team.find_elements_by_tag_name('td')[1].find_elements_by_tag_name('a')[0].text for team in teams]
        ranking = driver.find_elements_by_xpath('//*[@id="wettbewerb_head"]/div/div/div[2]/div[2]/div[2]/table/tbody/tr[1]/td/a').text
        for name in team_names:
            club_country[name] = key
    return club_country

In [ ]:
club_country = get_first_league_clubs2020()
club_country

In [ ]:
result_df = pd.DataFrame(club_country.items())
result_df.to_csv('clubs.tsv', sep='\t', index=False, header=False)